In [2]:
import pandas as pd

In [332]:
def main_preprocessor(name, congress=False):
    df = pd.read_csv(name + ".csv")
    df[["state_name", "state"]] = df.state.str.split(" - ",
                                                     expand=True)
    df = df.rename(columns={"District": "district"})
    if congress == True:
        df["district"] = df["district"].apply(district_at_large)
    return df

def preprocessor(name, feature):
    df = pd.read_csv(name + ".csv")
    df["name"] = df["name"].str.replace(r"^b'", "")
    df["name"] = df["name"].str.replace(r"'", "")
    df["name"] = df["name"].str.replace(r"\\xc3\\xa1", "a")
    df["name"] = df["name"].str.replace(r"\\xc3\\xb3", "o")
    df = df.rename(columns={"name": "Last Name"})
    df = df[["Last Name", feature, "state", "district"]]
    return df

def data_merge(main_df, ideology_df, leadership_df, criteria):
    df_final = pd.merge(main_df, ideology_df, on=criteria, how="left")
    df_final = pd.merge(df_final, leadership_df, on=criteria, how="left")
    return df_final[['ID', 'Year', 'First Name', 'Middle Name', 'Last Name', 'party',
                     'Level', 'Position', 'state', 'race_ethnicity',
                     'state_name', 'ideology','leadership', "district"]]

def district_at_large(dist):
    dist = str(dist)
    if dist == "AL":
        return 0
    else:
        dist = dist.replace("0", "")
        return int(dist)

In [339]:
#Senators Bio
senate = main_preprocessor("senate_women")
#Congress Bio
congress = main_preprocessor("congress_women", congress=True)
#Senate Ideology Scores
senate_ideology = preprocessor("senate_ideology", "ideology")
#Senate Leadership Scores
senate_leadership = preprocessor("senate_leadership", "leadership")
#merge
senate_women = data_merge(senate,
                          senate_ideology,
                          senate_leadership,
                          ["Last Name",
                           "state"])
senate_women = senate_women.fillna({"district": 0})
#congress ideology scores
congress_ideology = preprocessor("congress_ideology", "ideology")
#congress leadership scores
congress_leadership = preprocessor("congress_leadership", "leadership")
#merge
congress_women = data_merge(congress,
                            congress_ideology,
                            congress_leadership,
                            ["Last Name",
                             "state",
                             "district"])

        ID  Year First Name Middle Name  Last Name       party     Level  \
0  159127k  2022      Tammy         NaN    Baldwin    Democrat  Congress   
1  198986k  2022     Marsha         NaN  Blackburn  Republican  Congress   
2  205537k  2022      Maria         NaN   Cantwell    Democrat  Congress   
3  200084k  2022    Shelley       Moore     Capito  Republican  Congress   
4  165234k  2022      Susan         NaN    Collins  Republican  Congress   

       Position state district_x race_ethnicity     state_name  ideology  \
0  U.S. Senator    WI        S02          White      Wisconsin  0.209314   
1  U.S. Senator    TN        S02          White      Tennessee  1.000000   
2  U.S. Senator    WA        S02          White     Washington  0.341586   
3  U.S. Senator    WV        S02          White  West Virginia  0.755178   
4  U.S. Senator    ME        S01          White          Maine  0.608876   

   district_y  leadership  district  
0         NaN    0.655926       NaN  
1         

C:\Users\thiya\AppData\Local\Temp\ipykernel_27304\2808340490.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  df["name"] = df["name"].str.replace(r"^b'", "")
C:\Users\thiya\AppData\Local\Temp\ipykernel_27304\2808340490.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
  df["name"] = df["name"].str.replace(r"\\xc3\\xa1", "a")
C:\Users\thiya\AppData\Local\Temp\ipykernel_27304\2808340490.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df["name"] = df["name"].str.replace(r"\\xc3\\xb3", "o")
C:\Users\thiya\AppData\Local\Temp\ipykernel_27304\2808340490.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  df["name"] = df["name"].str.replace(r"^b'", "")
C:\Users\thiya\AppData\Local\Temp\ipykernel_27304\2808340490.py:27: FutureWarning: The default value of regex will change from True to F

In [340]:
twitter_handles = pd.read_csv("twitter_handles.csv")
twitter_handles = twitter_handles.rename(columns={"Name": "Full Name"})
twitter_handles = twitter_handles[["Full Name", "Link", "User_name"]]

In [341]:
women_in_office = pd.concat([senate_women, congress_women])
women_in_office["Middle Name"].fillna("", inplace=True)
women_in_office["Full Name"] = women_in_office["First Name"] + " " + women_in_office["Middle Name"] + " " + women_in_office["Last Name"]
women_in_office["Full Name"] = women_in_office["Full Name"].str.replace("  ", " ")
women_in_office = pd.merge(women_in_office, twitter_handles, on="Full Name")
len(women_in_office)

146